<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 5: Введение в ML-агенты

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/thumbnail.png" alt="Thumbnail" width="70%"/>

В этом блокноте вы узнаете о ML-агентах и обучите двух агентов.

- Первый из них научится **стрелять снежками по появляющимся мишеням**.
- Второму нужно нажать кнопку, чтобы создать пирамиду, затем перейти к пирамиде, опрокинуть ее **и перейти к золотому кирпичу наверху**. Чтобы сделать это, ему нужно будет исследовать свое окружение, и мы будем использовать технику, называемую любопытство (curiosity).

После этого вы сможете **наблюдать за игрой ваших агентов прямо в вашем браузере**.

⬇️ Вот пример того, чего **вы достигнете в конце этого блока.** ⬇️

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids.gif" alt="Pyramids"/>

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget.gif" alt="SnowballTarget" width="70%"/>

### 🎮 Окружающая среда: 

- Пирамиды [Pyramids](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#pyramids)
- Мишень для снежков (SnowballTarget)

### 📚 RL-Library: 

- [ML-Агенты (Экспериментальная версия с HuggingFace)](https://github.com/huggingface/ml-agents)

⚠ Мы собираемся использовать экспериментальную версию ML-Agents, где вы можете использовать hub и загружать из hub модели Unity ML-Agents **вам необходимо установить ту же версию**

## Цели этго блокнота 🏆

В конце блокнота вы:

- Поймете, как работает **ML-Agents** и библиотека окружения.
- Научитесь **обучать агентов в средах Unity**.


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="70%"/>

## Предварительные условия 🏗️
Прежде чем начать, вам нужно:

🔲  📚 ****Изучить [что такое ML-агенты и как это работает, прочитав блок 5](https://huggingface.co/deep-rl-course/unit5/introduction )** 🤗

# Давайте обучим наших агентов 🚀

Интеграция ML-агентов на хабе **все еще является экспериментальной**, некоторые функции будут добавлены в будущем.

Но на данный момент, **чтобы подтвердить это практическое руководство для процесса сертификации, вам просто нужно отправить ваши обученные модели в центр**. Нет никаких результатов, которых можно было бы достичь, чтобы подтвердить это. Но если вы хотите получить хорошие результаты, вы можете попытаться достичь:

- Для `Pyramids` : Среднюю награду= 1.75
- Для `SnowballTarget` : Среднюю награду = 15 или 30 целей, пораженных в эпизоде.

## Клонируйте репозиторий и установите зависимости 🔽
- Нам нужно клонировать репозиторий, который **содержит экспериментальную версию библиотеки, которая позволяет вам отправлять вашего обученного агента в концентратор.**

In [ ]:
%%capture
# Клонируйте репозиторий
!git clone --depth 1 https://github.com/huggingface/ml-agents/ 

In [ ]:
%%capture
# Зайдите в репозиторий и установите пакет
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

##Мишень для снежка ⛄

Если вам нужно уточнить, как работают эти среды, ознакомьтесь с этим разделом 👉
https://huggingface.co/deep-rl-course/unit5/snowball-target

### Загрузите и переместите zip-файл среды в `./training-envs-executables/linux/`
- Исполняемый файл нашей среды находится в zip-файле.
- Нам нужно загрузить его и поместить в `./training-envs-executables/linux/`
- Мы используем исполняемый файл linux, потому что мы используем colab, а ОС colab machines - Ubuntu (linux)

In [ ]:
# Здесь мы создаем папки training-envs-executables и linux
!mkdir ./training-envs-executables
!mkdir ./training-envs-executables/linux

Скачайте файл SnowballTarget.zip с https://drive.google.com/file/d/1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5 используя `wget`. 

Ознакомьтесь с полным решением для загрузки больших файлов из GDrive [здесь](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5" -O ./training-envs-executables/linux/SnowballTarget.zip && rm -rf /tmp/cookies.txt

Распакуем файл executable.zip

In [ ]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/SnowballTarget.zip

Убедитесь, что ваш файл доступен

In [ ]:
!chmod -R 755 ./training-envs-executables/linux/SnowballTarget

### Настройте конфигурационный файл SnowballTarget
- Для ML-Agents вы определяете гиперпараметры обучения **в файлах config.yaml.**

Существует несколько гиперпараметров. Чтобы узнать их лучше, вы должны изучить [документацию](https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Training-Configuration-File.md)


Итак, вам нужно создать`SnowballTarget.yaml` конфигурационный файл в папке ./content/ml-agents/config/ppo/

Мы предоставим вам здесь первую версию этой конфигурации (для копирования и вставки в ваш файл `SnowballTarget.yaml`), **но вы должны изменить его**.

```
behaviors:
  SnowballTarget:
    trainer_type: ppo
    summary_freq: 10000
    keep_checkpoints: 10
    checkpoint_interval: 50000
    max_steps: 200000
    time_horizon: 64
    threaded: true
    hyperparameters:
      learning_rate: 0.0003
      learning_rate_schedule: linear
      batch_size: 128
      buffer_size: 2048
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
    network_settings:
      normalize: false
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
```

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config1.png" alt="Config SnowballTarget" width="70%"/>
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config2.png" alt="Config SnowballTarget" width="70%"/>

В качестве эксперимента вам также следует попробовать изменить некоторые другие гиперпараметры. Unity предоставляет очень [хорошую документацию, объясняющую каждый из них здесь](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

Теперь, когда вы создали конфигурационный файл и понимаете, что делает большинство гиперпараметров, мы готовы обучить нашего агента 🔥.

### Обучите агента

Чтобы обучить нашего агента, нам просто нужно **запустить ml agents-learn и выбрать исполняемый файл, содержащий среду.**

Мы определяем четыре параметра:

1. `mlagents-learn <config>`: путь, в котором находится конфигурационный файл гиперпараметра.
2. `--env`: где находится исполняемый файл среды.
3. `--run_id`: имя, которое вы хотите присвоить своему идентификатору тренировочного прогона.
4. `--no-graphics`: чтобы не запускать визуализацию во время тренировки.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/mlagentslearn.png" alt="MlAgents learn" width="70%"/>

Обучите модель и используйте флаг `--resume`, чтобы продолжить обучение в случае прерывания. 

> В первый раз произойдет сбой, когда вы используете `--resume`, попробуйте запустить блок еще раз, чтобы обойти ошибку.

In [ ]:
!mlagents-learn ./config/ppo/SnowballTarget.yaml --env=./training-envs-executables/linux/SnowballTarget/SnowballTarget --run-id="SnowballTarget1" --no-graphics

Тренировка займет от 10 до 35 минут в зависимости от вашей конфигурации.

### Переместите агента в концентратор 🤗

- Теперь, когда мы обучили нашего агента, мы **готовы перенести его в центр, чтобы иметь возможность визуализировать его воспроизведение в вашем браузере 🔥 .**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Затем нам просто нужно запустить `mlagents-push-to-hf`.

И мы определяем 4 параметра:

1. `--run-id`: имя идентификатора тренировочного прогона.
2. `--local-dir`: где агент был сохранен, это `results/run_id name`, так что в моем случае `results/First Training`.
3. `--repo-id`: название репозитория Hugging Face, который вы хотите создать или обновить. Это всегда `your huggingface username/the repo name`
Если репозиторий не существует **, он будет создан автоматически**
4. `--commit-message`: поскольку репозитории HF являются репозиторием git, вам необходимо определить сообщение о фиксации.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/mlagentspushtohub.png" alt="Push to Hub" width="70%"/>

Например:

`!mlagents-push-to-hf  --run-id="SnowballTarget1" --local-dir="./results/SnowballTarget1" --repo-id="ThomasSimonini/ppo-SnowballTarget"  --commit-message="First Push"`

In [ ]:
!mlagents-push-to-hf  --run-id= # Добавить Ваш run id  --local-dir= # Ваша локальная папка  --repo-id= # Ваш repo id  --commit-message= # Ваше сообщение о фиксации

Если все сработало, вы должны получить это в конце процесса (но с другим URL 😆 ) :



```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-SnowballTarget
```

Это ссылка на вашу модель, она содержит карточку модели, которая объясняет, как ее использовать, вашу Tensorboard и ваш конфигурационный файл. **Что удивительно, так это то, что это репозиторий git, это означает, что вы можете иметь разные коммиты, обновлять свой репозиторий новым нажатием и т.д.**

Но теперь наступает самое лучшее: **возможность визуализировать своего агента онлайн 👀 .**

### Наблюдай за игрой своего агента 👀

Для этого шага необходимо:

1. Запомнить свой repo-id

2. Перейти: https://singularite.itch.io/snowballtarget

3. Запустите игру и переведите ее в полноэкранный режим, нажав на правую нижнюю кнопку

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget_load.png" alt="Snowballtarget load" width="70%"/>

1. На шаге 1 выберите репозиторий вашей модели, который является идентификатором модели (в моем случае `ThomasSimonini/ppo-SnowballTarget`).

2. На шаге 2 **выберите, какую модель вы хотите воспроизвести**:
   - У меня их несколько, так как мы сохраняли модель каждые 500000 временных шагов.
   - Но если я хочу более свежую версию, я выбираю `SnowballTarget.onnx'

👉 Что приятно **, так это попробовать с разными моделями шаг за шагом, чтобы увидеть улучшение агента.**

И не стесняйтесь делиться лучшим результатом, полученным вашим агентом на discord в #rl-i-made-this channel 🔥

Давайте теперь попробуем более сложную среду под названием Pyramids...

## Пирамиды (Pyramids) 🏆

### Загрузите и переместите zip-файл среды в `./training-envs-executables/linux/`
- Исполняемый файл нашей среды находится в zip-файле.
- Нам нужно загрузить его и разместить на `./training-envs-executables/linux/`
- Мы используем исполняемый файл linux, потому что мы используем colab, а ОС colab машины - это Ubuntu (linux)

Загрузите файл Pyramids.zip из https://drive.google.com/uc?export=download&id=1UiFNdKlsH0NTu32xV-giYUEVKV4-vc7H используя `wget`. Ознакомьтесь с полным решением для загрузки больших файлов из GDrive [здесь](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UiFNdKlsH0NTu32xV-giYUEVKV4-vc7H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UiFNdKlsH0NTu32xV-giYUEVKV4-vc7H" -O ./training-envs-executables/linux/Pyramids.zip && rm -rf /tmp/cookies.txt

**ИЛИ** Загрузите непосредственно на локальный компьютер, а затем перетащите файл с локального компьютера на `./training-envs-executables/linux`

Дождитесь завершения загрузки, а затем выполните приведенную ниже команду.

Распакуйте его

In [ ]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/Pyramids.zip

Убедитесь, что ваш файл доступен

In [ ]:
!chmod -R 755 ./training-envs-executables/linux/Pyramids/Pyramids

### Измените конфигурационный файл PyramidsRND
- В отличие от первой среды, которая была пользовательской, ** Pyramids была создана командой Unity**.
- - Итак, конфигурационный файл PyramidsRND уже существует и находится в ./content/ml-agents/config/ppo/PyramidsRND.yaml
- Вы могли бы спросить, что значит "RND". RND расшифровывается как *случайная сетевая дистилляция* это способ получения вознаграждения за любопытство. Если вы хотите узнать об этом больше, мы написали статью, объясняющую эту технику: https://medium.com/data-from-the-trenches/curiosity-driven-learning-through-random-network-distillation-488ffd8e5938

Для этого тренинга мы изменим одну вещь:
- Гиперпараметр "Общее количество шагов обучения" слишком высок, поскольку мы можем достичь контрольного показателя (среднее вознаграждение = 1,75) всего за 1 м шагов обучения.
👉 Чтобы сделать это, мы переходим в config/ppo/PyramidsRND.yaml,** и изменяем их на max_steps равным 1000000.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids-config.png" alt="Pyramids config" width="50%"/>

В качестве эксперимента вам также следует попробовать изменить некоторые другие гиперпараметры, Unity предоставляет очень [хорошую документацию, объясняющую каждый из них здесь](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

Теперь мы готовы обучить нашего агента🔥.

### Обучите агента

Тренировка займет от 30 до 45 минут в зависимости от вашего тренажера.

In [ ]:
!mlagents-learn ./config/ppo/PyramidsRND.yaml --env=./training-envs-executables/linux/Pyramids/Pyramids --run-id="Pyramids Training" --no-graphics

### Переместите агента в концентратор 🤗

- Теперь, когда мы обучили нашего агента, мы **готовы перенести его в центр, чтобы иметь возможность визуализировать его воспроизведение в вашем браузере 🔥 .**

### Watch your agent playing 👀

The temporary link for Pyramids demo is: https://singularite.itch.io/pyramids